In [1]:
import os
import sys
sys.path.append("../")

import panel as pn
import altair as alt
from altair import datum
import pandas as pd
# from vega_datasets import data
import datetime as dt

from src.utils.snowflake import read_forecast_meta, read_forecasts
from src.plot.altair import plot_all, plot_decompose

In [2]:
alt.renderers.enable("default")
pn.extension("vega")
pn.extension('tabulator')
# pn.extension(sizing_mode="stretch_width")

In [3]:
df_forecast_meta = read_forecast_meta()
df_table = (
    df_forecast_meta.rename(
        columns={
            "vermogen_nominaal": "P_nom",
            "relative_abs_max": "P_max",
            "date_abs_max": "t_max",
        }
    )
    [["boxid", "P_max", "t_max", "vestiging"]]
    .sort_values(by=["P_max", "t_max"], ascending=[False, True])
)
df_table["t_max"] = pd.to_datetime(df_table["t_max"]).dt.date
df_table

,boxid,P_max,t_max,vestiging
16,VRY.HOUBE-1,1.000000,2021-11-15,Venlo
1,063.691-1,0.875502,2021-12-13,Tilburg
9,159.628-1,0.827228,2021-12-13,Eindhoven
5,133.037-1,0.802911,2021-12-20,Breda
12,ESD.000363-1,0.778692,2022-02-14,Hengelo
17,VRY.MHANS-1,0.756287,2021-12-13,Venlo
14,ESD.000042-1,0.679398,2022-02-07,Hengelo
8,154.732-1,0.678712,2022-02-07,Den Bosch
4,121.633-1,0.667767,2021-12-27,Den Bosch
3,121.670-1,0.641002,2021-11-22,Den Bosch


In [24]:
df_table.iloc[3] = ["BRAM.VONK-1", 1.0, dt.datetime.now().date(), "Breda"]

In [33]:
df_table[["boxid", "P_max", "t_max"]].reset_index(drop=True)

,boxid,P_max,t_max
0,VRY.HOUBE-1,1.000000,2021-11-15
1,063.691-1,0.875502,2021-12-13
2,159.628-1,0.827228,2021-12-13
3,BRAM.VONK-1,1.000000,2021-11-19
4,ESD.000363-1,0.778692,2022-02-14
5,VRY.MHANS-1,0.756287,2021-12-13
6,ESD.000042-1,0.679398,2022-02-07
7,154.732-1,0.678712,2022-02-07
8,121.633-1,0.667767,2021-12-27
9,121.670-1,0.641002,2021-11-22


In [4]:
tabulator_formatters = {
    "float": {"type": "progress", "max": 1},
    "bool": {"type": "tickCross"},
}

table = pn.widgets.Tabulator(
    df_table.copy(),
#     formatters=tabulator_formatters,
    show_index=False,
    disabled=True,
    selectable=1,
    height=500,
    width=400,
    selection=[0],
    width_policy="min",
)

vestigingen = list(df_table["vestiging"].unique())
choice_vestiging = pn.widgets.CheckButtonGroup(
    name='regio', 
    value=vestigingen, 
    options=vestigingen,
    width=1100,
)

table.add_filter(choice_vestiging, "vestiging")

def update_tabs(selection):
    # extract boxid
    boxid = table.value["boxid"].iloc[selection].squeeze()
    
    df_total = read_forecasts(boxid=boxid)
    
    plot_total = plot_all(
        df_data=df_total, df_meta=df_forecast_meta[df_forecast_meta["boxid"]==boxid]
    ).properties(width=600, height=400)
    
    plot_min = plot_decompose(df_total[df_total["extreme"]=="min"])
    
    plot_max = plot_decompose(df_total[df_total["extreme"]=="max"])
    
    return pn.Tabs(("general", plot_total), ("breakdown min", plot_min), ("breakdown max", plot_max), width_policy="max")

tabs = pn.bind(update_tabs, selection=table.param.selection)
    
panel = pn.Column(choice_vestiging, pn.Row(table, tabs, min_height=500))
    
pn.serve(panel)

Launching server at http://localhost:52512


In [11]:
boxid = "063.691-1"